** Polymer stats iPython notebook**

Supplementary material for the IB Materials lectures on rubber elasticity and entropic forces.

Original version from A Kabla (2016). Extended to 3D by Hannan Saddiq, IB student (2018).

_______

This notebook provides a simple code to generate random polymer configurations in 3D and build the statistics of the end-to-end distance that is required for the calculation of the configurational entropy. Requires installation of the web plotting library plotly.

In [34]:
# Load relevant modules

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

import numpy as np

The function get_polymer_shape($n_m$,$a_m$) generates a random configuration for a polymer of $n_m$ monomers, each of length $a_m$. It returns an array of $x_i$ and $y_i$ coordinates for each monomer of the chain, with $x_0=y_0=0$.

In [4]:
def get_polymer_shape(nm, am):
    thetas = (np.pi) * np.random.rand(nm)
    phis = (2*np.pi) * np.random.rand(nm)
    dx = am * np.sin(thetas) * np.cos(phis)
    dy = am * np.sin(thetas) * np.sin(phis)
    dz = am * np.cos(thetas)
    
    x = np.add.accumulate(dx)
    y = np.add.accumulate(dy)
    z = np.add.accumulate(dz)
    return((x,y,z))


The cell below simply calculates and displays individual polymer configurations.

In [11]:
number_of_monomers = 100
monomer_length = 1.

(x,y,z)=get_polymer_shape(number_of_monomers, monomer_length)

trace = go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=4,
        color=(x**2 + y**2 + z**2)**0.5,
        colorscale='Viridis',
    ),
    line=dict(
        color='#1f77b4',
        width=1
    )
)

data = [trace]

layout = dict(
    width=800,
    height=700,
    autosize=False,
    title='3D Polymer Configuration',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        camera=dict(
            up=dict(
                x=0,
                y=0,
                z=1
            ),
            eye=dict(
                x=-1.7428,
                y=1.0707,
                z=0.7100,
            )
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    ),
)

fig = dict(data=data, layout=layout)

py.iplot(fig, filename='3D-polymer-configuration', height=700)


The function ensemble_stat calculates $n$ different polymer configurations of identical number of monomers $n_m$ and monomer length $a_m$, and returns 3 arrays containing the $x$ and $y$ coordinates of the end-to-end vectors, as well as the total length, of each configuration.

In [18]:
def ensemble_stat(n, nm, am):
    xa=np.zeros(n)
    ya=np.zeros(n)
    za=np.zeros(n)
    ra=np.zeros(n)
    for i in range(n):
        (x,y,z)=get_polymer_shape(nm, am)
        (xa[i],ya[i],za[i])=(x[-1],y[-1],z[-1])
        ra[i]=np.sqrt(xa[i]**2+ya[i]**2+za[i]**2)
    return(xa,ya,za,ra)


In [29]:
# number of polymer configurations
n = 5000

(xa,ya,za,ra) = ensemble_stat(n, number_of_monomers, monomer_length)

trace = go.Scatter3d(
    x=xa,
    y=ya,
    z=za,
    mode='markers',
    marker=dict(
        size=5,
        color=ra,                # set color to an array/list of desired values
        colorscale='Viridis',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
        aspectmode = "cube"
    )
)
fig = go.Figure(data=data, layout=layout)

py.iplot(fig, filename='end_to_end_distance')

In [36]:
x = go.Histogram(
    x=xa,
    nbinsx = int(np.sqrt(n)),
    name = "Histogram of x"
  )
y = go.Histogram(
    x=ya,
    nbinsx = int(np.sqrt(n)), 
        name = "Histogram of y"
  )
z = go.Histogram(
    x=za,
    nbinsx = int(np.sqrt(n)),
        name = "Histogram of z"
  )
r = go.Histogram(
    x=ra,
    nbinsx = int(np.sqrt(n)), 
        name = "Histogram of end to end distance"
  )

  
fig = tools.make_subplots(rows=2, cols=2)
fig.append_trace(x, 1, 1)
fig.append_trace(y, 1, 2)
fig.append_trace(z, 2, 1)
fig.append_trace(r, 2, 2)

py.iplot(fig, filename='xyz polymer histograms')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]  [ (2,2) x4,y4 ]



In [40]:
nm_list = [10,20,50,100,200,500,1000]
am_list = [1,2,5]
nval = len(nm_list)*len(am_list)

r = []
sqrtna = []

for n_m in nm_list:
    for a_m in am_list:
        (xa,ya,za,ra) = ensemble_stat(n, n_m, a_m)
        r.append(np.mean(ra))
        sqrtna.append(np.sqrt(n_m)*a_m)

# Add linear regression to the plot    
    
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(sqrtna,r)

print("<r>/sqrt(n)r = " + str(slope))

trace1 = go.Scatter(
    x = sqrtna,
    y = r,
    mode = 'markers'
)

trace2 = go.Scatter(
    x = [0,max(sqrtna)],
    y = [0, slope * max(sqrtna)],
    mode = 'lines'
)

data = [trace1,trace2]

layout = dict(
    showlegend=False,
    xaxis = dict(
        title="$ \sqrt{n} a$"
    ),
        yaxis = dict(
        title="$ r_g = \sqrt{< r^2 >}$"
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='polymer linear regression')

<r>/sqrt(n)r = 0.9223818920340726
